### Sleeping Barber with Monitors (Python) [6 points]

Python supports monitors through [Lock](https://docs.python.org/3/library/threading.html#lock-objects) objects and [Condition](https://docs.python.org/3/library/threading.html#condition-objects) objects. A lock is like a binary semaphore, but a lock is bound to the process (thread) that acquired the lock, meaning that only that process can release the lock. The pattern for implementing a monitor in Python is:

```Python
class M
    def __init__(self):
        self.lock = Lock()
        self.cond = Condition(self.lock)
        ...
    def m(self):
        with self.lock:
            ... self.cond.wait() ...
    def n(self):
        with self.lock:
            ... self.cond.signal() ...
            ... self.cond.signalAll() ...
```
The `with lock: ...` statement is equivalent to calling `lock.acquire()` at the beginning and `lock.release()` in a `finally` clause at the end – that is, the lock is released if an exception is raised or not; it corresponds to the `synchronized(lock) {...}` statement in Java. Like in C/Pthreads, condition objects have to be created with reference to a lock. Multiple conditions can refer to the same lock, as in:

```Python
    self.cond1, self.cond2 = Condition(self.lock), Condition(self.lock)
```

Implement the Sleeping Barber problem from the course notes with Python monitors! You may add output to trace the progress.

In [1]:
from threading import Thread, Lock, Condition
from sys import stdout

class BarberShop:
    def __init__(self):
        self.barber = 0; self.chair = 0; self.exit = 0
        self.lock = Lock()
        self.barberAvailable = Condition(self.lock)
        self.chairOccupied = Condition(self.lock)
        self.exitOpen = Condition(self.lock)
        self.customerLeft = Condition(self.lock)
        
    def getHaircut(self, i):
        with self.lock:
            while self.barber == 0: 
                stdout.write(f"{i} is waiting for the barber.\n")
                self.barberAvailable.wait()
            self.barber -= 1
            self.chair += 1
            stdout.write(f"{i} is sitting in the chair.\n")
            self.chairOccupied.notify()
            while self.exit == 0:
                stdout.write(f"{i} is getting a haircut.\n")
                self.exitOpen.wait()
            self.exit -= 1
            stdout.write(f"{i} finished the haircut.\n")
            self.customerLeft.notify()
            
    def getNextCustomer(self):
        with self.lock:
            self.barber += 1
            stdout.write("Barber is getting up.\n")
            self.barberAvailable.notify()
            while self.chair == 0:
                stdout.write("Barber is waiting for a customer to sit.\n")
                self.chairOccupied.wait()
            self.chair -= 1
            
    def finishedCut(self):
        with self.lock:
            self.exit += 1
            stdout.write("Barber is waiting for the customer to leave.\n")
            self.exitOpen.notify()
            while self.exit > 0: self.customerLeft.wait()
            

class Barber(Thread):
    def __init__(self, shop):
        Thread.__init__(self); self.shop = shop
    def run(self):
        for _ in range(20):
            self.shop.getNextCustomer()  # wait for a customer to sit in the barber's chair
            stdout.write("barber cutting hair\n")
            self.shop.finishedCut()      # allow the customer to leave; returns after the customer left

class Customer(Thread):
    def __init__(self, i, shop):
        Thread.__init__(self); self.i, self.shop = i, shop
    def run(self):
        for _ in range(5):
            stdout.write(str(self.i) + " living happily\n")
            self.shop.getHaircut(self.i)  # returns after the customer has received a the haircut


s = BarberShop(); b = Barber(s)
c0 = Customer(0, s); c1 = Customer(1, s); c2 = Customer(2, s); c3 = Customer(3, s)
b.start(); c0.start(); c1.start(); c2.start(); c3.start()

Barber is getting up.
Barber is waiting for a customer to sit.
0 living happily
0 is sitting in the chair.
0 is getting a haircut.
barber cutting hair
Barber is waiting for the customer to leave.
0 finished the haircut.
0 living happily
0 is waiting for the barber.
Barber is getting up.
Barber is waiting for a customer to sit.
0 is sitting in the chair.
0 is getting a haircut.
1 living happily
1 is waiting for the barber.
barber cutting hair
Barber is waiting for the customer to leave.
0 finished the haircut.
0 living happily
0 is waiting for the barber.
Barber is getting up.
Barber is waiting for a customer to sit.
2 living happily
2 is sitting in the chair.
2 is getting a haircut.
barber cutting hair
Barber is waiting for the customer to leave.
1 is waiting for the barber.
2 finished the haircut.
2 living happily
2 is waiting for the barber.
3 living happily
3 is waiting for the barber.
Barber is getting up.
Barber is waiting for a customer to sit.
0 is sitting in the chair.
0 is get